<a href="https://colab.research.google.com/github/Juliadambros/Aplicacoes-Inteligencia-Artificial/blob/main/Aplica%C3%A7%C3%A3o_de_Algoritmo_Gen%C3%A9tico_para_Recomenda%C3%A7%C3%A3o_de_Filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configurações



In [ ]:
import random

TAXA_CRUZAMENTO = 0.8
TAXA_MUTACAO = 0.1
TAMANHO_POPULACAO = 100
NUM_GERACOES = 50
TAMANHO_LISTA = 5

preferencias = {
    "genero": ["ação", "ficção"],
    "duracao_ideal": 120,
    "nota_minima": 7.0
}



Banco de Filmes

In [ ]:
filmes_possiveis = [
    {"titulo": "Filme A", "genero": "ação", "duracao": 110, "nota": 8.0, "ano": 2018},
    {"titulo": "Filme B", "genero": "drama", "duracao": 130, "nota": 6.5, "ano": 2019},
    {"titulo": "Filme C", "genero": "ficção", "duracao": 125, "nota": 8.7, "ano": 2021},
    {"titulo": "Filme D", "genero": "comédia", "duracao": 95, "nota": 7.1, "ano": 2020},
    {"titulo": "Filme E", "genero": "ação", "duracao": 140, "nota": 8.3, "ano": 2016},
    {"titulo": "Filme F", "genero": "romance", "duracao": 105, "nota": 6.9, "ano": 2017},
    {"titulo": "Filme G", "genero": "ficção", "duracao": 118, "nota": 9.0, "ano": 2022},
    {"titulo": "Filme H", "genero": "ação", "duracao": 100, "nota": 7.5, "ano": 2020},
    {"titulo": "Filme I", "genero": "terror", "duracao": 90, "nota": 5.8, "ano": 2019},
    {"titulo": "Filme J", "genero": "ficção", "duracao": 130, "nota": 8.9, "ano": 2023},
    {"titulo": "Filme K", "genero": "ação", "duracao": 115, "nota": 9.2, "ano": 2021},
    {"titulo": "Filme L", "genero": "aventura", "duracao": 125, "nota": 7.8, "ano": 2015},
    {"titulo": "Filme M", "genero": "ficção", "duracao": 145, "nota": 8.4, "ano": 2017},
    {"titulo": "Filme N", "genero": "ação", "duracao": 135, "nota": 7.9, "ano": 2020},
    {"titulo": "Filme O", "genero": "ficção", "duracao": 105, "nota": 9.1, "ano": 2024},
    {"titulo": "Filme P", "genero": "drama", "duracao": 150, "nota": 8.2, "ano": 2019},
    {"titulo": "Filme Q", "genero": "ficção", "duracao": 90, "nota": 7.4, "ano": 2022},
    {"titulo": "Filme R", "genero": "ação", "duracao": 125, "nota": 8.1, "ano": 2020},
    {"titulo": "Filme S", "genero": "terror", "duracao": 100, "nota": 6.0, "ano": 2016},
    {"titulo": "Filme T", "genero": "ação", "duracao": 112, "nota": 8.5, "ano": 2023}
]


Funções do Algoritmo

In [ ]:
def gerar_cromossomo():
    return random.sample(range(len(filmes_possiveis)), TAMANHO_LISTA)


def calcular_fitness(cromossomo):
    pontuacao = 0
    for i in cromossomo:
        filme = filmes_possiveis[i]
        # Preferência de gênero
        if filme["genero"] in preferencias["genero"]:
            pontuacao += 3
        # Duração próxima da ideal
        pontuacao += max(0, 2 - abs(filme["duracao"] - preferencias["duracao_ideal"]) / 60)
        # Nota mínima
        if filme["nota"] >= preferencias["nota_minima"]:
            pontuacao += (filme["nota"] - preferencias["nota_minima"]) * 1.5
    return round(pontuacao, 2)


def selecao_roleta(populacao):
    total_fitness = sum(c["fitness"] for c in populacao)
    if total_fitness == 0:
        return random.choice(populacao)
    pick = random.uniform(0, total_fitness)
    atual = 0
    for c in populacao:
        atual += c["fitness"]
        if atual >= pick:
            return c


def cruzamento(pai1, pai2):
    if random.random() > TAXA_CRUZAMENTO:
        return pai1.copy(), pai2.copy()
    ponto = random.randint(1, TAMANHO_LISTA - 2)
    filho1 = pai1[:ponto] + [g for g in pai2 if g not in pai1[:ponto]]
    filho2 = pai2[:ponto] + [g for g in pai1 if g not in pai2[:ponto]]
    return filho1[:TAMANHO_LISTA], filho2[:TAMANHO_LISTA]


def mutacao(cromossomo):
    if random.random() < TAXA_MUTACAO:
        i, j = random.sample(range(TAMANHO_LISTA), 2)
        cromossomo[i], cromossomo[j] = cromossomo[j], cromossomo[i]
    return cromossomo




In [ ]:
def executar_geracao(populacao, geracao_atual):
    nova_populacao = []
    while len(nova_populacao) < TAMANHO_POPULACAO:
        pai1 = selecao_roleta(populacao)["genes"]
        pai2 = selecao_roleta(populacao)["genes"]
        filho1, filho2 = cruzamento(pai1, pai2)
        filho1 = mutacao(filho1)
        filho2 = mutacao(filho2)
        nova_populacao.append({"genes": filho1, "fitness": calcular_fitness(filho1)})
        nova_populacao.append({"genes": filho2, "fitness": calcular_fitness(filho2)})

    melhor = max(nova_populacao, key=lambda c: c["fitness"])
    filmes_melhor = [filmes_possiveis[i]["titulo"] for i in melhor["genes"]]

    print(f"\n=== GERAÇÃO {geracao_atual} ===")
    print(f"Melhor fitness: {melhor['fitness']}")
    print("Filmes recomendados:", ", ".join(filmes_melhor))

    return nova_populacao, melhor


In [ ]:
# POPULAÇÃO INICIAL (parte fixa + aleatória)

populacao_inicial_manual = [
    [0, 2, 6, 10, 7],  # Exemplo 1: vários de ação/ficção
    [3, 4, 9, 11, 13], # Exemplo 2: mistura de gêneros
]

# Completa com cromossomos aleatórios até o tamanho desejado
populacao = [{"genes": c} for c in populacao_inicial_manual]
while len(populacao) < TAMANHO_POPULACAO:
    populacao.append({"genes": gerar_cromossomo()})

# Calcula fitness inicial
for individuo in populacao:
    individuo["fitness"] = calcular_fitness(individuo["genes"])

melhor_resultado = []

In [ ]:
# EXECUÇÃO
for geracao in range(1, NUM_GERACOES + 1):
    populacao, melhor = executar_geracao(populacao, geracao)
    melhor_resultado.append({
        "geracao": geracao,
        "fitness": melhor["fitness"],
        "filmes": [filmes_possiveis[i]["titulo"] for i in melhor["genes"]]
    })



=== GERAÇÃO 1 ===
Melhor fitness: 37.17
Filmes recomendados: Filme C, Filme J, Filme T, Filme M, Filme K

=== GERAÇÃO 2 ===
Melhor fitness: 37.58
Filmes recomendados: Filme J, Filme C, Filme O, Filme R, Filme G

=== GERAÇÃO 3 ===
Melhor fitness: 37.58
Filmes recomendados: Filme O, Filme J, Filme R, Filme G, Filme C

=== GERAÇÃO 4 ===
Melhor fitness: 35.17
Filmes recomendados: Filme T, Filme J, Filme C, Filme R, Filme A

=== GERAÇÃO 5 ===
Melhor fitness: 35.07
Filmes recomendados: Filme N, Filme H, Filme O, Filme J, Filme G

=== GERAÇÃO 6 ===
Melhor fitness: 33.8
Filmes recomendados: Filme E, Filme M, Filme G, Filme H, Filme T

=== GERAÇÃO 7 ===
Melhor fitness: 35.98
Filmes recomendados: Filme J, Filme N, Filme A, Filme O, Filme G

=== GERAÇÃO 8 ===
Melhor fitness: 34.2
Filmes recomendados: Filme C, Filme E, Filme J, Filme N, Filme A

=== GERAÇÃO 9 ===
Melhor fitness: 35.83
Filmes recomendados: Filme C, Filme E, Filme J, Filme N, Filme G

=== GERAÇÃO 10 ===
Melhor fitness: 37.63
Filmes

In [ ]:
print("\n=== RESULTADOS FINAIS ===")
for r in melhor_resultado:
    print(f"Geração {r['geracao']:2d} | Fitness: {r['fitness']:5.2f} | Filmes: {', '.join(r['filmes'])}")



=== RESULTADOS FINAIS ===
Geração  1 | Fitness: 37.17 | Filmes: Filme C, Filme J, Filme T, Filme M, Filme K
Geração  2 | Fitness: 37.58 | Filmes: Filme J, Filme C, Filme O, Filme R, Filme G
Geração  3 | Fitness: 37.58 | Filmes: Filme O, Filme J, Filme R, Filme G, Filme C
Geração  4 | Fitness: 35.17 | Filmes: Filme T, Filme J, Filme C, Filme R, Filme A
Geração  5 | Fitness: 35.07 | Filmes: Filme N, Filme H, Filme O, Filme J, Filme G
Geração  6 | Fitness: 33.80 | Filmes: Filme E, Filme M, Filme G, Filme H, Filme T
Geração  7 | Fitness: 35.98 | Filmes: Filme J, Filme N, Filme A, Filme O, Filme G
Geração  8 | Fitness: 34.20 | Filmes: Filme C, Filme E, Filme J, Filme N, Filme A
Geração  9 | Fitness: 35.83 | Filmes: Filme C, Filme E, Filme J, Filme N, Filme G
Geração 10 | Fitness: 37.63 | Filmes: Filme C, Filme E, Filme J, Filme O, Filme G
Geração 11 | Fitness: 37.63 | Filmes: Filme C, Filme E, Filme J, Filme O, Filme G
Geração 12 | Fitness: 37.63 | Filmes: Filme C, Filme E, Filme J, Filme 